In [6]:
import string
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [7]:
# Path to translation file
path_to_data = 'D:/MyData/Cours/DM/Data/fra-eng/fra.txt'

# Read file
translation_file = open(path_to_data,"r", encoding='utf-8') 
raw_data = translation_file.read()
translation_file.close()

# Parse data
raw_data = raw_data.split('\n')
pairs = [sentence.split('\t') for sentence in  raw_data]
pairs = pairs[10000:20000]

In [8]:
#nettoyer le corpus
def clean_sentence(sentence):
    # Lower case the sentence
    lower_case_sent = sentence.lower()
    # Strip punctuation
    string_punctuation = string.punctuation + "¡" + '¿'
    clean_sentence = lower_case_sent.translate(str.maketrans('', '', string_punctuation))
   
    return clean_sentence

In [9]:
# tokenizer le corpus
def tokenize(sentences):
    # Create tokenizer
    text_tokenizer = Tokenizer()
    # Fit texts
    text_tokenizer.fit_on_texts(sentences)
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer

In [10]:
# Clean sentences
english_sentences = [clean_sentence(pair[0]) for pair in pairs]
french_sentences = [clean_sentence(pair[1]) for pair in pairs]

# Tokenize words
fra_text_tokenized, fra_text_tokenizer = tokenize(french_sentences)
eng_text_tokenized, eng_text_tokenizer = tokenize(english_sentences)

print('Maximum length french sentence: {}'.format(len(max(fra_text_tokenized,key=len))))
print('Maximum length english sentence: {}'.format(len(max(eng_text_tokenized,key=len))))

Maximum length french sentence: 11
Maximum length english sentence: 5


In [11]:
# Check language length
french_vocab = len(fra_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1
print("French vocabulary is of {} unique words".format(french_vocab))
print("English vocabulary is of {} unique words".format(english_vocab))

French vocabulary is of 5357 unique words
English vocabulary is of 2736 unique words


In [17]:
#padding
max_french_len = int(len(max(fra_text_tokenized,key=len)))
max_english_len = int(len(max(fra_text_tokenized,key=len)))

fra_pad_sentence = pad_sequences(fra_text_tokenized, max_french_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_french_len, padding = "post")

# Reshape data
fra_pad_sentence = fra_pad_sentence.reshape(*fra_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

print('Maximum length french sentence: {}'.format(fra_pad_sentence.shape))
print('Maximum length english sentence: {}'.format(eng_pad_sentence.shape))

Maximum length french sentence: (10000, 11, 1)
Maximum length english sentence: (10000, 11, 1)


In [18]:
# creation du model
#Encoder
input_sequence = Input(shape=(max_french_len,))
embedding = Embedding(input_dim=french_vocab, output_dim=128,)(input_sequence)
encoder = LSTM(64, return_sequences=False)(embedding)

In [19]:
#Decoder
r_vec = RepeatVector(max_english_len)(encoder)
decoder = LSTM(64, return_sequences=True, dropout=0.2)(r_vec)
logits = TimeDistributed(Dense(english_vocab))(decoder)

In [20]:
enc_dec_model = Model(input_sequence, Activation('softmax')(logits))
enc_dec_model.compile(loss=sparse_categorical_crossentropy,
              optimizer=Adam(1e-3),
              metrics=['accuracy'])
enc_dec_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 11)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 11, 128)           685696    
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 11, 64)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 11, 64)            33024     
                                                                 
 time_distributed_1 (TimeDis  (None, 11, 2736)         177840    
 tributed)                                                 

In [21]:
model_results = enc_dec_model.fit(fra_pad_sentence, eng_pad_sentence, batch_size=30, epochs=100)

Epoch 1/100
334/334 [==============================] - 25s 46ms/step - loss: 2.6456 - accuracy: 0.7003
Epoch 2/100
334/334 [==============================] - 14s 42ms/step - loss: 1.8578 - accuracy: 0.7186
Epoch 3/100
334/334 [==============================] - 13s 39ms/step - loss: 1.7925 - accuracy: 0.7218
Epoch 4/100
334/334 [==============================] - 15s 44ms/step - loss: 1.7572 - accuracy: 0.7228
Epoch 5/100
334/334 [==============================] - 15s 44ms/step - loss: 1.6876 - accuracy: 0.7291
Epoch 6/100
334/334 [==============================] - 14s 40ms/step - loss: 1.6291 - accuracy: 0.7345
Epoch 7/100
334/334 [==============================] - 12s 35ms/step - loss: 1.5844 - accuracy: 0.7372
Epoch 8/100
334/334 [==============================] - 13s 39ms/step - loss: 1.5348 - accuracy: 0.7396
Epoch 9/100
334/334 [==============================] - 15s 44ms/step - loss: 1.4864 - accuracy: 0.7424
Epoch 10/100
334/334 [==============================] - 11s 33ms/step - l

In [22]:
def logits_to_sentence(logits, tokenizer):

    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>' 

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

In [31]:
index_vocab = [8, 14, 100]
for index in index_vocab:
    print("vacob : {} ".format(index, index))
    print("The english sentence is: {}".format(english_sentences[index]))
    print("The french sentence is: {}".format(french_sentences[index]))
    print('The predicted sentence is :')
    print(logits_to_sentence(enc_dec_model.predict(fra_pad_sentence[index:index+1])[0], eng_text_tokenizer))

vacob : 8 
The english sentence is: get on the bus
The french sentence is: monte dans le bus 
The predicted sentence is :
1/1 [==============================] - 0s 33ms/step
get on the bus <empty> <empty> <empty> <empty> <empty> <empty> <empty>
vacob : 14 
The english sentence is: get rid of tom
The french sentence is: débarrassezvous de tom
The predicted sentence is :
1/1 [==============================] - 0s 46ms/step
get rid of tom <empty> <empty> <empty> <empty> <empty> <empty> <empty>
vacob : 100 
The english sentence is: he disappeared
The french sentence is: il a filé
The predicted sentence is :
1/1 [==============================] - 0s 48ms/step
he disappeared <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty> <empty>
